In [ ]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

MODELS_TO_KEEP = [
    "ChemBertMLM-10M",
    "ChemBertMTR-77M",
    "ChemGPT-1.2B",
    "ContextPred",
    "GraphMVP",
    "GROVER",
    "GraphLog",
    "GraphCL",
    "InfoGraph",
    "FRAD_QM9",
    "MolR_gat",
    "ThreeDInfomax",
]

DATASET_NEW_NAME = {
    "hERG": "hERG",
    "hERG_Karim": "hERG (k)",
    "AMES": "AMES",
    "DILI": "DILI",
    "Carcinogens_Lagunin": "Carcinogens",
    "Skin__Reaction": "Skin R",
    "Tox21": "Tox21",
    "ClinTox": "ClinTox",
    "LD50_Zhu": "LD50",
    "PAMPA_NCATS": "PAMPA",
    "HIA_Hou": "HIA",
    "Pgp_Broccatelli": "Pgp",
    "Bioavailability_Ma": "Bioavailability",
    "Caco2_Wang": "Caco2",
    "Lipophilicity_AstraZeneca": "Lipophilicity",
    "Solubility_AqSolDB": "Solubility",
    "HydrationFreeEnergy_FreeSolv": "FreeSolv",
    "BBB_Martins": "BBB",
    "PPBR_AZ": "PPBR",
    "VDss_Lombardo": "VDss",
    "CYP2C19_Veith": "CYP2C19",
    "CYP2D6_Veith": "CYP2D6",
    "CYP3A4_Veith": "CYP3A4",
    "CYP1A2_Veith": "CYP1A2",
    "CYP2C9_Veith": "CYP2C9",
    "CYP2C9_Substrate_CarbonMangels" : "CYP2C9 (s)",
    "CYP2D6_Substrate_CarbonMangels" : "CYP2D6 (s)",
    "CYP3A4_Substrate_CarbonMangels" : "CYP3A4 (s)",
    "Half_Life_Obach" : "Half Life",
    "Clearance_Hepatocyte_AZ" : "Clearance (H)",
    "Clearance_Microsome_AZ" : "Clearance (M)",
}

In [ ]:
df_baselines = pd.read_csv('../baslines.csv')[["embedder", "dataset", "metric_test"]]
df_distill = pd.read_csv('../model_95.csv')[["embedder", "dataset", "metric_test"]]
df_distill["embedder"] = "distill"

df_baselines = df_baselines[df_baselines.embedder.isin(MODELS_TO_KEEP)]
df_or = pd.concat([df_baselines, df_distill]).groupby(["dataset", "embedder"]).metric_test.mean().reset_index()
df = df_or.pivot_table(index="embedder", columns="dataset", values="metric_test")
df.dropna(axis=1, inplace=True)
# drop column and index names
df.index.name = None
order = df.mean(axis=1).sort_values(ascending=False).index

df.mean(axis=1).sort_values(ascending=False)



DATASET_NEW_NAME = {d: DATASET_NEW_NAME[d] for d in DATASET_NEW_NAME if d in df.columns}
df = df.rename(columns=DATASET_NEW_NAME)
df = df[DATASET_NEW_NAME.values()]
df = df.loc[order[::-1],:]
df.index = df.index.str.replace("_", " ")

df

In [ ]:
# Bold the best value for each dataset
def highlight_max(s):
    is_max = s == s.max()
    return ['font-weight: bold' if v else '' for v in is_max]

#Render with everything in 5pt font
style = df.style.format("{:.3f}").highlight_max(axis=0, props='bfseries:')


In [ ]:
latex = style.to_latex()

with open("table.tex", "w") as f:
    f.write(latex)